<a href="https://colab.research.google.com/github/mahendrapd/GRST/blob/main/GRST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
data = pd.read_parquet('/content/drive/MyDrive/dataset/training.parquet')

In [4]:
cols=len(data.columns)
nfeatures = cols-1
X = data.iloc[:,0:nfeatures]
y = data.iloc[:,-1]
print(len(data),cols)

347122 55


In [5]:
targetclass='Benign'

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [7]:
def RSTClassifier(X,y,targetclass=y[0]):
    print(targetclass)
    dc=pd.Series(y)
    cm=dc.groupby(dc).apply(lambda px: px.index.tolist()).to_dict()
    D=set(cm[targetclass])
    #print(D)
    purity={}
    headers = X.keys().tolist()

    for j in range(nfeatures):
        header=headers[j]
        hdata={}
        #print("Columns:",j,header)
        dp=pd.Series(X.iloc[:,j])
        md=dp.groupby(dp).apply(lambda px: px.index.tolist()).to_dict()
        K=md.keys()
        #print(K)
        for z in K:
            common_elements=set(md[z]) & D
            val=round(len(common_elements)/len(md[z]),3)
            #print(z,len(md[z]),len(common_elements),val)
            hdata[z]=val
        purity[header]=hdata
    #print(purity['urgent'][0])
    return purity

In [8]:
pur=RSTClassifier(X_train,y_train,'Benign')

Benign


In [9]:
print(pur)

{'Flow Duration': {0.0: 1.0, 0.1: 1.0, 0.24: 1.0, 0.26: 1.0, 0.3: 1.0, 0.37: 1.0, 0.4: 1.0, 0.42: 1.0, 0.7: 1.0, 0.88: 1.0, 0.92: 1.0, 0.99: 1.0, 1.0: 0.911}, 'Total Fwd Packets': {0.0: 0.911, 0.01: 0.973, 0.02: 0.959, 0.03: 0.955, 0.04: 1.0, 0.05: 0.889, 0.06: 1.0, 0.07: 0.75, 0.08: 0.5, 0.09: 0.0, 0.1: 0.0, 0.11: 0.75, 0.12: 0.0, 0.13: 0.0, 0.14: 1.0, 0.15: 0.0, 0.17: 0.0, 0.19: 1.0, 0.2: 0.0, 0.21: 0.0, 0.22: 0.0, 0.23: 0.0, 0.24: 0.0, 0.25: 0.0, 0.26: 0.0, 0.27: 0.5, 0.28: 0.333, 0.29: 0.0, 0.3: 0.0, 0.32: 0.0, 0.33: 0.0, 0.34: 0.0, 0.35: 0.5, 0.36: 0.333, 0.37: 1.0, 0.39: 0.5, 0.4: 0.5, 0.41: 0.0, 0.43: 0.0, 0.45: 0.5, 0.46: 1.0, 0.48: 0.0, 0.49: 0.0, 0.5: 0.5, 0.51: 0.333, 0.52: 0.667, 0.53: 0.5, 0.54: 0.5, 0.56: 0.5, 0.57: 0.0, 0.58: 0.75, 0.59: 0.5, 0.6: 1.0, 0.61: 0.5, 0.62: 0.5, 0.63: 1.0, 0.64: 1.0, 0.65: 0.667, 0.66: 1.0, 0.67: 1.0, 0.68: 0.5, 0.69: 0.5, 0.7: 1.0, 0.71: 0.667, 0.73: 0.0, 0.76: 0.0, 0.78: 0.0, 0.79: 0.0, 0.8: 0.0, 0.84: 0.0, 0.9: 0.0}, 'Total Backward Packet

In [10]:
def RST_AmbigiousSample(X,pur):
    headers = X.keys().tolist()
    print(headers,len(headers),len(X))
    ambigioussamples={}
    for i in range(len(X)):
        flag=0
        sumpurity=0
        for j in range(len(headers)):
            key=X.iloc[i][headers[j]]
            if (pur[headers[j]][key] == 1 or pur[headers[j]][key] == 0):
                flag=1
                break
            else:
                sumpurity+=pur[headers[j]][key]
        if (flag == 0):
            Avgpurity=round(sumpurity/len(headers),3)
            ambigioussamples[i]=Avgpurity
    return ambigioussamples

In [11]:
AmbSample=RST_AmbigiousSample(X_train,pur)

['Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Fwd Packets Length Total', 'Bwd Packets Length Total', 'Fwd Packet Length Max', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Max', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'Avg Packet Size', 'Avg Fwd Segment Size', 'Avg Bwd Segment Size', 'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Packets', 'Subflow Bwd Bytes', 'Init Fwd Win Bytes', 'Init Bwd Win Bytes', 'Fwd Act Data Packets', 'Fwd Seg Size Min', 'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle 

In [12]:
print(len(AmbSample))

241632


In [13]:
def RST_Gradient(y,sample,epoch=1000):
    Theta=0.5
    LRate=0.01

    for i in range(epoch):
        correctcount=0
        val=0
        Th=Theta
        for key, value in sample.items():
            if (value >= Theta and y.iloc[key]==targetclass):
                correctcount+=1
            elif (value < Theta and y.iloc[key]!=targetclass):
                correctcount+=1
            else:
                val+=value
        ictotal=(len(sample)-correctcount)
        Theta=round(Theta-2*LRate*(Theta-round((val/ictotal),3)),3)
        print(i, correctcount, ictotal, len(sample), Theta)
    return Theta

In [14]:
Th=RST_Gradient(y_train,AmbSample,1000)

0 219403 22229 241632 0.508
1 219403 22229 241632 0.515
2 219403 22229 241632 0.522
3 219403 22229 241632 0.529
4 219403 22229 241632 0.536
5 219403 22229 241632 0.543
6 219403 22229 241632 0.55
7 219403 22229 241632 0.557
8 219403 22229 241632 0.563
9 219403 22229 241632 0.569
10 219403 22229 241632 0.575
11 219403 22229 241632 0.581
12 219403 22229 241632 0.587
13 219403 22229 241632 0.593
14 219403 22229 241632 0.599
15 219403 22229 241632 0.605
16 219403 22229 241632 0.61
17 219403 22229 241632 0.615
18 219403 22229 241632 0.62
19 219403 22229 241632 0.625
20 219403 22229 241632 0.63
21 219403 22229 241632 0.635
22 219403 22229 241632 0.64
23 219403 22229 241632 0.645
24 219403 22229 241632 0.65
25 219403 22229 241632 0.655
26 219403 22229 241632 0.659
27 219403 22229 241632 0.663
28 219403 22229 241632 0.667
29 219403 22229 241632 0.671
30 219403 22229 241632 0.675
31 219403 22229 241632 0.679
32 219403 22229 241632 0.683
33 219403 22229 241632 0.687
34 219403 22229 241632 0.691
3

In [15]:
def RST_Validation(X,y,pur,Theta):
    TP=0
    TN=0
    FP=0
    FN=0
    headers = X.keys().tolist()
    for i in range(len(X)):
        val=0
        count=0
        pval=0

        flag=0
        for j in range(len(headers)):
            #KYS=list(pur[headers[j]].keys())
            #dflag=0
            key=X.iloc[i][headers[j]]
            try:
              pval=pur[headers[j]][key]
            except KeyError:
              pval=Theta

            if ((pval == 1 or pval == 0)):
                flag=1
                break
            else:
                val+=pval
                count+=1

        if (flag == 1):
            if (pval==1 and y.iloc[i]==targetclass):
                TP+=1
            elif (pval==1 and y.iloc[i]!=targetclass):
                FN+=1
            elif (pval==0 and y.iloc[i]!=targetclass):
                TN+=1
            else:
                FP+=1
        else:
            Aval=round((val/count),3)
            if (Aval>=Theta and y.iloc[i]==targetclass):
                TP+=1
            elif (Aval<Theta and y.iloc[i]!=targetclass):
                TN+=1
            elif (Aval>=Theta and y.iloc[i]!=targetclass):
                FN+=1
            else:
                FP+=1
    Recall=round(TP/(TP+FN),4)
    FPR=round(FP/(TN+FP),4)
    Precision=round(TP/(TP+FP),4)
    Accuracy=round((TP+TN)/(TP+TN+FP+FN),4)
    FScore=round(2*Recall*Precision/(Recall+Precision),4)
    print("TP=",TP,"\tTN=",TN,"\tFP=",FP,"\tFN=",FN)
    return Recall, FPR, Precision, FScore, Accuracy

In [16]:
Result=RST_Validation(X_train,y_train,pur,Th)
print(Result)

TP= 236576 	TN= 7127 	FP= 588 	FN= 16050
(0.9365, 0.0762, 0.9975, 0.966, 0.9361)


In [17]:
ResultTest = RST_Validation(X_test,y_test,pur,Th)
print(ResultTest)

TP= 78857 	TN= 2393 	FP= 209 	FN= 5322
(0.9368, 0.0803, 0.9974, 0.9662, 0.9363)


In [20]:
def CountPatterns(pur):
  for outer_key, outer_value in pur.items():
    acount=0
    ncount=0
    for inner_key, inner_value in outer_value.items():
      if (inner_value == 0):
        acount=acount+1
      if (inner_value == 1):
        ncount=ncount+1
    print(f"{outer_key}: {acount}, {ncount}, {len(outer_value.items())}")

In [21]:
CountPatterns(pur)

Flow Duration: 0, 12, 13
Total Fwd Packets: 30, 12, 71
Total Backward Packets: 0, 46, 50
Fwd Packets Length Total: 0, 4, 12
Bwd Packets Length Total: 0, 33, 36
Fwd Packet Length Max: 0, 30, 46
Fwd Packet Length Mean: 6, 17, 46
Fwd Packet Length Std: 1, 17, 44
Bwd Packet Length Max: 3, 12, 31
Bwd Packet Length Mean: 1, 5, 17
Bwd Packet Length Std: 4, 4, 32
Flow Bytes/s: 39, 3, 90
Flow Packets/s: 5, 5, 39
Flow IAT Mean: 0, 4, 5
Flow IAT Std: 0, 11, 12
Flow IAT Max: 0, 13, 14
Flow IAT Min: 0, 9, 10
Fwd IAT Total: 0, 12, 13
Fwd IAT Mean: 0, 4, 5
Fwd IAT Std: 0, 11, 12
Fwd IAT Max: 0, 13, 14
Fwd IAT Min: 0, 9, 10
Bwd IAT Total: 0, 0, 101
Bwd IAT Mean: 0, 14, 101
Bwd IAT Std: 0, 7, 101
Bwd IAT Max: 0, 2, 101
Bwd IAT Min: 0, 27, 101
Fwd Header Length: 82, 0, 90
Bwd Header Length: 0, 7, 8
Fwd Packets/s: 10, 1, 46
Bwd Packets/s: 0, 15, 34
Packet Length Max: 0, 25, 49
Packet Length Mean: 0, 10, 22
Packet Length Std: 0, 11, 32
Packet Length Variance: 0, 7, 12
Avg Packet Size: 0, 10, 24
Avg Fwd Se